# INNOMATICS Data Analysis and Machine Learning HACKATHON

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
data=pd.read_excel('uber_rides_data.xlsx')

In [3]:
# Load first five rows
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
# Load last five rows
data.tail()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
199995,42598914,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1
199999,11951496,14.1,2010-05-15 04:08:00 UTC,-73.984395,40.720077,-73.985508,40.768793,1


In [5]:
# Check statistics of data
data.describe()

,ride_id,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+05,200000.000000,200000.000000,200000.000000,199999.000000,199999.000000,200000.000000
mean,2.771250e+07,11.359955,-72.527638,39.935885,-72.525292,39.923890,1.684535
std,1.601382e+07,9.901776,11.437787,7.720539,13.117408,6.794829,1.385997
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,1.382535e+07,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,2.774550e+07,8.500000,-73.981823,40.752592,-73.980093,40.753042,1.000000
75%,4.155530e+07,12.500000,-73.967153,40.767158,-73.963658,40.768001,2.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [6]:
# Check shape of data
data.shape

(200000, 8)

In [7]:
# Check information of data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [8]:
# check missing values
data['dropoff_longitude'].isnull().sum()

1

In [9]:
# Drop values
data = data.dropna()

In [10]:
# check missing values
data.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [11]:
data.fare_amount.mean()

11.359891549458371

In [12]:
data.fare_amount.max()

499.0

In [13]:
import math
def haversine_dist(row):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1 = math.radians(row['pickup_latitude']), math.radians(
        row['pickup_longitude'])
    lat2, lon2 = math.radians(row['dropoff_latitude']), math.radians(
        row['dropoff_longitude'])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * \
        math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return distance

In [14]:
data['haversine_dist'] = data.apply(haversine_dist, axis=1)

In [15]:
#calculating the median distance
median_distance = np.median(data['haversine_dist'])
print("Median Haversine Distance:", median_distance, "kilometers")

Median Haversine Distance: 2.120992396182902 kilometers


In [16]:
# counting the numbers of rides with a Haversine distance as zeroes
zero_distance_ride = data[data['haversine_dist'] == 0]
print(len(zero_distance_ride))

5632


In [17]:
# Calculate the mean 'fare_amount' for rides with 0.0 Haversine Distance
mean_fare_for_zero_distance_rides = zero_distance_ride['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine Distance:",
      mean_fare_for_zero_distance_rides)

Mean 'fare_amount' for rides with 0.0 Haversine Distance: 11.585317826704578


In [18]:
import math

# Define the Haversine distance function


def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * \
        math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return distance


# Find the ride with the highest 'fare_amount'
costliest_ride = data[data['fare_amount'] == data['fare_amount'].max()]

# Calculate the Haversine distance for the costliest ride
haversine_dist_costliest_ride = calculate_haversine_distance(costliest_ride.iloc[0]['pickup_latitude'],
                                                             costliest_ride.iloc[0]['pickup_longitude'],
                                                             costliest_ride.iloc[0]['dropoff_latitude'],
                                                             costliest_ride.iloc[0]['dropoff_longitude'])

print("Haversine Distance for the Costliest Ride:",
      haversine_dist_costliest_ride, "per kilometers")

Haversine Distance for the Costliest Ride: 0.0007899213191009994 per kilometers


In [19]:
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_dist
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,4.475450


In [20]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], errors='coerce')

In [21]:
# Creating a  separate columns for year, month, and week
data['year'] = data['pickup_datetime'].dt.year
data['month'] = data['pickup_datetime'].dt.month
data['week'] = data['pickup_datetime'].dt.week

In [22]:
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_dist,year,month,week
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,2015,5,19
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,2009,7,29
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,2009,8,35
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,2009,6,26
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,2014,8,35


In [23]:
ride_2014 = data[data['year']==2014]
print(len(ride_2014))

29968


In [24]:
ride_q1_2014 = data[(data['year']==2014) & (data['month']>=1) & (data['month']<=3)]
print(len(ride_q1_2014))

7687


In [25]:
ride_sept_2010 = data[(data['year'] == 2010) & (data['month'] == 9)]
ride_sept_2010['day_of_week'] = ride_sept_2010['pickup_datetime'].dt.day_name()
rides_by_day = ride_sept_2010['day_of_week'].value_counts()
# Find the day with the maximum recorded rides
max_rides_day = rides_by_day.idxmax()
max_rides_count = rides_by_day.max()

print("On which day of the week in September 2010, maximum rides were recorded?")
print("Day:", max_rides_day)
print("Number of rides:", max_rides_count)

On which day of the week in September 2010, maximum rides were recorded?
Day: Thursday
Number of rides: 457


In [26]:
# importing more libraries

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [27]:
data.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'haversine_dist', 'year', 'month', 'week'],
      dtype='object')

In [28]:
# splitting the data into X and y
X = data[['passenger_count', 'haversine_dist', 'week']]
y = data['fare_amount']

In [29]:
X.head()

,passenger_count,haversine_dist,week
0,1,1.683323,19
1,1,2.457590,29
2,1,5.036377,35
3,3,1.661683,26
4,5,4.475450,35


In [30]:
y.head()

0     7.5
1     7.7
2    12.9
3     5.3
4    16.0
Name: fare_amount, dtype: float64

In [31]:
# splitting the data for test and train in (30-70)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
# building the model 
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'KNN Regression': KNeighborsRegressor()
}

In [33]:
adjusted_r2_scores = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate R-squared
    r2 = r2_score(y_test, y_pred)

    # Calculate adjusted R-squared
    n = X_test.shape[0]  # Number of samples
    p = X_test.shape[1]  # Number of predictors
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    adjusted_r2_scores[model_name] = adjusted_r2

In [34]:
# Finding the model with the least adjusted R-squared
worst_model = min(adjusted_r2_scores, key=adjusted_r2_scores.get)

print("Model with the least adjusted R-squared:", worst_model)

Model with the least adjusted R-squared: Linear Regression
